# Pytorch Quantization

PyTorch mendukung kuantisasi INT8 dibandingkan dengan model FP32 yang umum sehingga memungkinkan pengurangan 4x dalam ukuran model dan pengurangan 4x dalam persyaratan bandwidth memori
sambil tetap mencapai akurasi yang sebanding untuk banyak aplikasi. Buku catatan ini menunjukkan cara mengkuantisasi model dari FP32 ke INT8 menggunakan perkakas kuantisasi PyTorch. Kami akan melatih model CNN sederhana pada mnist dan kemudian mengkuantisasinya menggunakan perkakas kuantisasi dan membandingkan akurasi dan ukuran model yang dikuantisasi dengan model FP32 asli.

## Siapkan PyTorch

Pertama, mari instal PyTorch dan torchvision, lalu impor modul yang diperlukan.

In [11]:
%pip install torch torchvision

In [12]:
import torch #Mengimpor pustaka PyTorch, yang digunakan untuk membangun dan melatih model deep learning.
import torch.nn as nn # Mengimpor modul `nn` dari PyTorch, yang menyediakan berbagai lapisan dan fungsi untuk membangun jaringan saraf.
import torch.nn.functional as F # Mengimpor fungsi fungsional dari PyTorch, yang berisi berbagai fungsi aktivasi, fungsi loss, dll.
import torch.optim as optim # Mengimpor modul `optim` dari PyTorch, yang menyediakan algoritma optimisasi untuk memperbarui bobot model.
from torchvision import datasets, transforms # Mengimpor `datasets` dan `transforms` dari `torchvision` untuk memudahkan pengolahan gambar dan dataset.
import torch.quantization # Mengimpor modul kuantisasi dari PyTorch untuk mengoptimalkan model dengan mengurangi presisi data.
import pathlib # Mengimpor pustaka `pathlib` untuk memanipulasi dan bekerja dengan path file dan direktori.

## Kuantisasi Dinamis

Untuk kuantisasi dinamis, bobot dikuantisasi tetapi aktivasi dibaca atau disimpan dalam floating point dan aktivasi hanya dikuantisasi untuk komputasi.

### Memuat dataset MNIST

Pertama, kita memuat dataset MNIST

In [13]:
# Membuat pipeline transformasi yang akan diterapkan pada data gambar
transform = transforms.Compose([
        transforms.ToTensor(),  # Mengubah gambar menjadi tensor PyTorch. Gambar akan dikonversi ke format tensor dengan dimensi (C, H, W), yaitu channel, tinggi, dan lebar
        transforms.Normalize((0.1307,), (0.3081,))  # Menormalkan gambar dengan rata-rata (0.1307) dan deviasi standar (0.3081), yang sesuai dengan dataset MNIST
        ])

# Memuat dataset MNIST untuk pelatihan. Data akan disimpan di direktori './data', dan jika belum ada, dataset akan diunduh. Transformasi yang telah didefinisikan akan diterapkan pada data pelatihan
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)

# Memuat dataset MNIST untuk pengujian (test). Transformasi yang sama akan diterapkan pada data pengujian.
test_dataset = datasets.MNIST('./data', train=False, transform=transform)


### Melatih Model

Selanjutnya, kami mendefinisikan model CNN sederhana dan kemudian melatihnya pada dataset MNIST

In [14]:
# Mendefinisikan kelas model neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Layer konvolusi pertama, dengan input 1 channel (grayscale image), output 12 channel, dan kernel 3x3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3)
        # Layer pooling maksimum dengan ukuran kernel 2x2 dan stride 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Layer Fully Connected (FC) untuk output 10 kelas (untuk klasifikasi digit MNIST)
        self.fc = nn.Linear(12 * 13 * 13, 10)

    def forward(self, x):
        # Mengubah bentuk input menjadi 4 dimensi (-1 adalah batch size yang otomatis ditentukan, 1 adalah jumlah channel, 28x28 adalah ukuran gambar)
        x = x.reshape(-1, 1, 28, 28)
        # Menjalankan input melalui layer konvolusi pertama dan aktivasi ReLU
        x = F.relu(self.conv1(x))
        # Menjalankan input melalui layer pooling
        x = self.pool(x)
        # Mengubah input menjadi bentuk vektor satu dimensi sebelum dimasukkan ke layer FC
        x = x.reshape(x.size(0), -1)
        # Melalui layer FC dan menghasilkan output untuk 10 kelas
        x = self.fc(x)
        # Menggunakan log softmax untuk menghasilkan probabilitas log untuk setiap kelas
        output = F.log_softmax(x, dim=1)
        return output


# Membuat DataLoader untuk data pelatihan dan pengujian, dengan ukuran batch 32
train_loader = torch.utils.data.DataLoader(train_dataset, 32)
test_loader = torch.utils.data.DataLoader(test_dataset, 32)

# Memilih device untuk pelatihan (GPU jika tersedia, jika tidak CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Menetapkan jumlah epoch
epochs = 1

# Menginisialisasi model, optimizer, dan memindahkan model ke device yang telah dipilih
model = Net().to(device)
optimizer = optim.Adam(model.parameters())

# Mengubah model ke mode pelatihan
model.train()

# Loop untuk setiap epoch
for epoch in range(1, epochs+1):
    # Loop untuk setiap batch dalam DataLoader pelatihan
    for batch_idx, (data, target) in enumerate(train_loader):
        # Memindahkan data dan target ke device yang telah dipilih
        data, target = data.to(device), target.to(device)

        # Mengatur gradien optimizer menjadi nol
        optimizer.zero_grad()

        # Melakukan forward pass untuk mendapatkan output dari model
        output = model(data)

        # Menghitung loss menggunakan Negative Log-Likelihood Loss (NLLLoss)
        loss = F.nll_loss(output, target)

        # Melakukan backward pass untuk menghitung gradien
        loss.backward()

        # Memperbarui parameter model dengan optimizer
        optimizer.step()

        # Menampilkan progres pelatihan (epoch, batch, dan loss)
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.276172
Train Epoch: 1 [32/60000 (0%)]	Loss: 2.261181
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.167956
Train Epoch: 1 [96/60000 (0%)]	Loss: 1.994696
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.061158
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.018412
Train Epoch: 1 [192/60000 (0%)]	Loss: 1.694208
Train Epoch: 1 [224/60000 (0%)]	Loss: 1.790973
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.864609
Train Epoch: 1 [288/60000 (0%)]	Loss: 1.572603
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.622503
Train Epoch: 1 [352/60000 (1%)]	Loss: 1.482278
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.471231
Train Epoch: 1 [416/60000 (1%)]	Loss: 1.308674
Train Epoch: 1 [448/60000 (1%)]	Loss: 1.218686
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.577137
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.396695
Train Epoch: 1 [544/60000 (1%)]	Loss: 1.127923
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.395177
Train Epoch: 1 [608/60000 (1%)]	Loss: 1.378385
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.363495
Train Epoch: 1 [67

### Kuantisasi Model

Setelah pelatihan, kita dapat mengkuantisasi model menggunakan fungsi `torch.quantization.quantize_dynamic` dari pytorch.

In [15]:
# Memindahkan model ke CPU
model.to('cpu')

# Melakukan quantization dinamis pada model, dengan mengubah tipe data layer Linear menjadi 8-bit integer (qint8)
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)


### Periksa Ukuran Model

Kita dapat melihat bahwa model terkuantisasi jauh lebih kecil daripada model asli

In [16]:
# Menentukan path direktori untuk menyimpan model
models_dir = pathlib.Path("./models/")
models_dir.mkdir(exist_ok=True, parents=True)

# Menyimpan model yang telah dilatih (model asli) ke dalam file dengan ekstensi .p
torch.save(model.state_dict(), "./models/original_model.p")

# Menyimpan model yang sudah melalui proses quantization ke dalam file dengan ekstensi .p
torch.save(quantized_model.state_dict(), "./models/quantized_model.p")

# Menampilkan daftar isi dari direktori "models" beserta ukuran file
%ls -lh models


total 136K
-rw-r--r-- 1 root root 82K Jan  4 11:08 original_model.p
-rw-r--r-- 1 root root 25K Jan  4 10:50 post_quantized_model.p
-rw-r--r-- 1 root root 23K Jan  4 11:08 quantized_model.p


### Periksa Akurasi

Kita dapat melihat bahwa model terkuantisasi memiliki akurasi yang sebanding dengan model asli

In [17]:
# Fungsi untuk menguji akurasi model
def test(model, device, data_loader, quantized=False):
    # Memindahkan model ke device (misalnya CPU atau GPU)
    model.to(device)
    # Menetapkan model ke mode evaluasi (non-training)
    model.eval()

    test_loss = 0  # Untuk menyimpan total kerugian
    correct = 0  # Untuk menghitung jumlah prediksi yang benar

    # Nonaktifkan perhitungan gradient (untuk menghemat memori dan mempercepat inference)
    with torch.no_grad():
        # Loop untuk setiap batch data di data_loader
        for data, target in data_loader:
            # Memindahkan data dan target ke device yang sama dengan model
            data, target = data.to(device), target.to(device)
            # Melakukan prediksi dengan model
            output = model(data)
            # Menghitung kerugian menggunakan Negative Log Likelihood Loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # Menjumlahkan kerugian per batch
            # Menentukan prediksi dengan nilai probabilitas terbesar
            pred = output.argmax(dim=1, keepdim=True)
            # Menghitung jumlah prediksi yang benar
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Menghitung rata-rata kerugian per sampel
    test_loss /= len(data_loader.dataset)

    # Mengembalikan akurasi dalam persen
    return 100. * correct / len(data_loader.dataset)

# Menguji akurasi model asli
original_acc = test(model, "cpu", test_loader)
# Menguji akurasi model yang telah diquantize
quantized_acc = test(quantized_model, "cpu", test_loader)

# Menampilkan hasil akurasi dari kedua model
print('Original model accuracy: {:.0f}%'.format(original_acc))
print('Quantized model accuracy: {:.0f}%'.format(quantized_acc))


Original model accuracy: 97%
Quantized model accuracy: 97%


## Kuantisasi Statis Pasca-pelatihan

Kuantisasi statis pasca-pelatihan adalah saat bobot dan aktivasi dikuantisasi dan kalibrasi diperlukan pasca-pelatihan. Di sini, kami mengkuantisasi model menggunakan fungsi `torch.quantization.quantize_fx()` dari PyTorch dan membandingkan akurasi dan ukuran model yang dikuantisasi dengan model FP32 asli.

Untuk mengkuantisasi menggunakan alat kuantisasi statis pasca-pelatihan, pertama-tama tentukan model atau muat model yang telah dilatih sebelumnya, lalu buat pemetaan konfigurasi kuantisasi menggunakan default untuk mesin QNNPACK. Atur model ke mode evaluasi dan buat tensor masukan sampel. Kemudian, persiapkan model untuk kuantisasi menggunakan fungsi `quantize_fx.prepare_fx()`. Ini melibatkan penerapan pemetaan konfigurasi kuantisasi dan persiapan model untuk menangani presisi int8. Model yang disiapkan kemudian dieksekusi pada tensor masukan. Terakhir, model terkuantisasi dengan memanggil `quantize_fx.convert_fx()` dan menyimpan model ke disk.

In [18]:
# Mengimpor modul yang diperlukan untuk kuantisasi
from torch.ao.quantization import (
  get_default_qconfig_mapping,
  get_default_qat_qconfig_mapping,
  QConfigMapping,
)
import torch.ao.quantization.quantize_fx as quantize_fx
import copy

# Memuat model yang telah dilatih sebelumnya
loaded_model = Net()
loaded_model.load_state_dict(torch.load("./models/original_model.p"))
model_to_quantize = copy.deepcopy(loaded_model)

# Mendapatkan konfigurasi kuantisasi default untuk 'qnnpack' (algoritma kuantisasi yang digunakan)
qconfig_mapping = get_default_qconfig_mapping("qnnpack")

# Menetapkan model ke mode evaluasi
model_to_quantize.eval()

# Mengambil input FP32 pertama dari data test untuk mempersiapkan model untuk kuantisasi
input_fp32 = next(iter(test_loader))[0][0:1]
input_fp32.to('cpu')  # Memindahkan input ke CPU

# Mempersiapkan model untuk kuantisasi (membuat versi model yang siap untuk kuantisasi)
model_fp32_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, input_fp32)

# Menjalankan model yang sudah dipersiapkan untuk kuantisasi dengan input FP32
model_fp32_prepared(input_fp32)

# Mengonversi model ke format INT8 setelah kuantisasi
model_int8 = quantize_fx.convert_fx(model_fp32_prepared)

# Menyimpan model yang telah dikuantisasi ke dalam file
torch.save(model_int8.state_dict(), "./models/post_quantized_model.p")


<ipython-input-18-01b6b1aec99a>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load("./models/original_model.p"))


## Periksa Ukuran Model

Sekali lagi, kita dapat melihat bahwa model terkuantisasi jauh lebih kecil daripada model asli

In [19]:
%ls -lh models

total 136K
-rw-r--r-- 1 root root 82K Jan  4 11:08 original_model.p
-rw-r--r-- 1 root root 25K Jan  4 11:08 post_quantized_model.p
-rw-r--r-- 1 root root 23K Jan  4 11:08 quantized_model.p


## Periksa Akurasi

Sekali lagi, kita dapat melihat bahwa akurasi model terkuantisasi tidak jauh berbeda dengan akurasi aslinya

In [20]:
# Menguji model yang telah dikuantisasi
quantized_acc = test(model_int8, "cpu", test_loader, quantized=True)

# Mencetak akurasi model yang telah dikuantisasi
print('Post quantized model accuracy: {:.0f}%'.format(quantized_acc))

Post quantized model accuracy: 97%
